<a href="https://colab.research.google.com/github/Akasxh/Stock_Analysis/blob/main/sec_edgar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
pip install beautifulsoup4

In [38]:
pip install sec-edgar-downloader

In [39]:
pip install openpyxl

In [40]:
pip install arelle

In [41]:
from sec_edgar_downloader import Downloader

In [42]:
import os
import glob
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from datetime import datetime

In [43]:
# Initialize the downloader
dl = Downloader("YOUR_COMPANY_NAME", "YOUR_EMAIL_ADDRESS","/content")

In [46]:
def extract_text_from_html(file_path):
    # Ensure the file exists
    if not os.path.exists(file_path):
        return "File does not exist"

    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract text from the parsed HTML
    text = soup.get_text(separator='\n', strip=True)

    return text

In [52]:
def truncate_and_overwrite(file_path, target_line):
    """
    Truncates the file content after a specific target line and overwrites the original file.

    :param file_path: Path to the file to be modified.
    :param target_line: The line after which all text should be discarded.
    """
    # Read the content and truncate after the target line
    with open(file_path, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()
    target_line = ' XBRL INSTANCE DOCUMENT'
    # Determine the point to truncate
    for i, line in enumerate(lines):
        if target_line in line:
            lines = lines[:i+1]  # Include the target line and stop after
            break

    # Overwrite the original file with truncated content
    with open(file_path, 'w', encoding='utf-8') as outfile:
        outfile.writelines(lines)






In [47]:
def merge(company_ticker):
    # Assuming dl.get() is correctly imported and functional
    dl.get("10-K", company_ticker.upper(), after="1995-01-01", before="2023-01-01", download_details=False)

    # Define the base directory
    base_directory = '/content'

    # List and sort subdirectories by modification time, latest first
    subdirectories = [os.path.join(base_directory, name) for name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, name))]
    subdirectories.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    latest_directory = subdirectories[0]  # Select the latest directory

    # Process all text files in the latest directory
    file_paths = [os.path.join(latest_directory, file) for file in os.listdir(latest_directory) if file.endswith('.txt')]
    # Open one file for all merged content
    output_filename = f'/content/merged_output_{company_ticker.upper()}.txt'

    #print(extract_text_from_html("/content/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/full-submission.txt"))



    with open(output_filename, 'w') as output_file:

      for file_path in file_paths:
        truncate_and_overwrite(file_path)
        extracted_data = extract_text_from_html(file_path)
        output_file.write(extracted_data + '\n')  # Write data to the file with a newline



In [48]:
def visualise(company_ticker):
    merge(company_ticker)

In [49]:
def main():
    company_ticker = input("Enter the company ticker: ")
    visualise(company_ticker)

In [53]:
main()

Enter the company ticker: AAPL


In [ ]:
from IPython.display import display, HTML

# Specify the path to your HTML file
file_path = '/content/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/full-submission.txt'

# Open and read the HTML file
with open(file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Display the HTML content in the notebook
display(HTML(html_content))